# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Jonathan Marcuse

**ID**: jrm564

> **Due Date**
>
> Friday, 10/13/23, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [68]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Library/CloudStorage/OneDrive-CornellUniversity/Cornell Onedrive/Cornell Desktop/Fall2023/BEE4750/lab03-jrmarcuse`


In [69]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver
using Plots # plotting

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

For an example of using `JuMP.jl` to solve linear programs, see [the
relevant tutorial on the class
website](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html).

Free free to delete some of the illustrative cells and code blocks in
your notebook as you go through and solve the lab problems…this might
help reduce some potential confusion while grading about what your
answer is.

## Introduction

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
\$0.12 per bf to purchase and fir costs \$0.08 per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board. This resource
allocation problem is diagrammed in
<a href="#fig-schematic" class="quarto-xref">Figure 1</a>.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

<figure id="fig-schematic">
<img src="attachment:lab03_files/figure-ipynb/mermaid-figure-1.png" />
<figcaption>Figure 1: Flowchat of the resource allocation problem in
this lab.</figcaption>
</figure>

## Problems (10 points)

### Problem 1: Problem Formulation (5 points)

In this problem, you will go through the steps of formulating a linear
program for this problem.

#### Problem 1.1 (1 point)

What are your decision variables? Clearly define your notation,
including what variables you are using, what they mean, and what their
units are.

My decision variable are the amount of Spruce and the amount of Fir to wood that will be associated with each plywood grade of production. Spruce towards grade 1 plywood will be variable S[1], Spruce towards grade 2 will be S[2] and towards grade 3 will be S[3]. Fir for plywood grade 1 will be variable F[1], Fir for grade 2 plywood will be F[2] and for grade 3 it will be F[3] and they are in units of bf. All variables must be greater than or equal to 0 as you cannot purchase a negative amount of wood.

In [70]:
#First I will define the wood model
wood_model = Model(HiGHS.Optimizer)

#Next I will define the Spruce (S) and Fir (F) variables
T=1:3
@variable(wood_model,S[t in T] >= 0)
@variable(wood_model,F[t in T] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{VariableRef}:
 F[1]
 F[2]
 F[3]

In [71]:
all_variables(wood_model)

6-element Vector{VariableRef}:
 S[1]
 S[2]
 S[3]
 F[1]
 F[2]
 F[3]

#### Problem 1.2 (1 point)

Derive your objective function. Support your function with
justifications and/or equations as necessary. You will not receive
credit just for the function alone.

The objective function will be profits which involves adding up all the revenue from plywood sales and then subtracting the costs of purchasing the raw materials.

0.5 bf of spruce and 1.5 bf of fir makes 1bf of grade 1 plywood, so therefore 500 bf of S and 1500 bf of F makes $400.

For grade 1 for every 2000 bf in total it would be $400

For grade 2 for every 3000 bf total it makes $520

For grade 3 for every 3500 bf it makes $700

In [72]:
@objective(wood_model, Max,400*((S[1]+F[1])/2000)
+520*((S[2]+F[2])/3000)
+700*((S[3]+F[3])/3500)
-sum(S)*0.12-sum(F)*0.08)

0.08000000000000002 S[1] + 0.12000000000000001 F[1] + 0.053333333333333344 S[2] + 0.09333333333333334 F[2] + 0.08000000000000002 S[3] + 0.12000000000000001 F[3]

#### Problem 1.3 (2 point)

Derive any needed constraints. Support your function with justifications
and/or equations as necessary. You will not receive credit just for the
final constraints alone.

Constraints will include the total amount of Spruce and Fir wood that is available to purchase from the woods. Other constraints include fixing the ratios of spruce to fir for each plywood grade making process.

In [73]:
@constraint(wood_model,Spruce_Resources,sum(S)<=320000)
@constraint(wood_model,Fir_Resources,sum(F)<=720000)
@constraint(wood_model,Grade1_ratio, F[1]==3*S[1])
@constraint(wood_model,Grade2_ratio, F[2]==2*S[2])
@constraint(wood_model,Grade3_ratio, F[3]==(4/3)*S[3])


Grade3_ratio : -1.3333333333333333 S[3] + F[3] = 0

#### Problem 1.4 (1 point)

Put this optimization problem in mathematical programming form. For an
example of the syntax for this, see lines 82–91
[here](https://github.com/vsrikrish/environmental-systems-analysis/blob/Fall23/tutorials/julia-jump.qmd).

In [ ]:
print(latex_formulation(wood_model))

$$ \begin{aligned}
\max\quad & 0.08000000000000002 S_{1} + 0.12000000000000001 F_{1} + 0.053333333333333344 S_{2} + 0.09333333333333334 F_{2} + 0.08000000000000002 S_{3} + 0.12000000000000001 F_{3}\\
\text{Subject to} \quad & -3 S_{1} + F_{1} = 0\\
 & -2 S_{2} + F_{2} = 0\\
 & -1.3333333333333333 S_{3} + F_{3} = 0\\
 & S_{1} + S_{2} + S_{3} \leq 320000\\
 & F_{1} + F_{2} + F_{3} \leq 720000\\
 & S_{1} \geq 0\\
 & S_{2} \geq 0\\
 & S_{3} \geq 0\\
 & F_{1} \geq 0\\
 & F_{2} \geq 0\\
 & F_{3} \geq 0\\
\end{aligned} $$

In [75]:
optimize!(wood_model)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 3 cols, 6 nonzeros
2 rows, 3 cols, 6 nonzeros
Presolve : Reductions: rows 2(-3); columns 3(-3); elements 6(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.4666614604e-01 Ph1: 2(4.58333); Du: 3(0.446666) 0s
          2    -1.1200000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.01


In [76]:
value.(S)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
 176000.00000000003
     -0.0
 143999.99999999997

In [77]:
value.(F)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
 528000.0000000001
      0.0
 191999.99999999994

In [78]:
objective_value(wood_model)

112000.00000000001

### Problem 2: Find the Solution (5 points)

#### Problem 2.1 (2 points)

Code your linear program using `JuMP`. Feel free to consult [the
website’s `JuMP`
tutorial](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html)
for syntax help. The keys:

1.  Initialize your model with a solver; in this case, we’ll use the
    `HiGHS` solver, but there are other solvers listed here for
    different types of problems, some of which are open and some of
    which require a commercial license:
    <https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers>:

    ``` julia
    example_model = Model(HiGHS.Optimizer)
    ```

2.  Define variables with syntax like

    ``` julia
    @variable(example_model, 1 >= example_x >= 0)
    ```

    This will create a variable `example_x` which is constrained to be
    between 0 and 1; you can leave off any of the bounds if a variable
    is unbounded in a particular direction. You can also add a vector of
    variables:

    ``` julia
    T = 1:3 # define set to index variables
    @variable(example_model, 1 >= example_z[t in T] >= 0)
    ```

    which will create a vector of 3 variables `example_z[1]`, …,
    `example_z[3]`, all of which are bounded between 0 and 1.

3.  Add an objective with

    ``` julia
    @objective(example_model, Max, example_x + sum(example_z))
    ```

    which will add an objective to maximize (replace with `Min` to
    minimize).

4.  Add constraints:

    ``` julia
    @constraint(example_model, constraint1, 2example_x + 3*sum(example_z) <= 10)
    @constraint(example_model, constraint2, 5example_x - example_z[1] <= 2)
    ```

    which will name the constraints `constraint1` and `constraint2` (you
    should make yours more descriptive about what the constraint
    actually is). The value of adding a name is to facilitate later
    querying of shadow prices, which we will discuss later. You can also
    add a vector of constraints which have similar structure or rely on
    different elements of a data vector:

    ``` julia
    A = [2; 4]
    b = [8; 12]
    I = 1:2 # set indices for constraint
    @constraint(example_model, vector_constraint[i in I], A[i] * sum(example_z) .<= b[i])
    ```

    You can also define matrices of constraints which depend on two
    index sets by generalizing this syntax, e.g.

    ``` julia
    @constraint(example_model, matrix_constraint[i in I, j in J, ...])
    ```

    > **Tip**
    >
    > Specifying higher-dimensional vectors and matrices of variables
    > and constraints will be important when we start looking at more
    > complex applications, so don’t skip over this! You don’t want to
    > manually enter thousands of constraints to ensure hourly
    > electricity demand is met…

    Finally, you can (and probably should) `print` your model to make
    sure that you get something that looks like the equations that you
    wrote down (in a notebook, this will be nicely rendered):

    ``` julia
    print(example_model)
    ```

    $$ \begin{aligned}
    \max\quad & example\_x + example\_z_{1} + example\_z_{2} + example\_z_{3}\\
    \text{Subject to} \quad & 2 example\_x + 3 example\_z_{1} + 3 example\_z_{2} + 3 example\_z_{3} \leq 10\\
     & 5 example\_x - example\_z_{1} \leq 2\\
     & 2 example\_z_{1} + 2 example\_z_{2} + 2 example\_z_{3} \leq 8\\
     & 4 example\_z_{1} + 4 example\_z_{2} + 4 example\_z_{3} \leq 12\\
     & example\_x \geq 0\\
     & example\_z_{1} \geq 0\\
     & example\_z_{2} \geq 0\\
     & example\_z_{3} \geq 0\\
     & example\_x \leq 1\\
     & example\_z_{1} \leq 1\\
     & example\_z_{2} \leq 1\\
     & example\_z_{3} \leq 1\\
    \end{aligned} $$

    > **Define your entire model in one cell**
    >
    > `JuMP` has great and intuitive syntax, but it doesn’t like
    > re-defining variables or constraints once they’ve been set. I
    > recommend putting all of your model-definition code (starting from
    > `model = Model(...)`) for a particular optimization problem in a
    > single notebook cell, so you can re-set up the entire problem with
    > a single click when you want to make a change.

#### Problem 2.2 (1 points)

Find the solution to your program and find the optimal values of the
decision variables. Once you’ve defined your model, you can find the
solution with \`optimize!():

In [79]:
optimize!(example_model)

UndefVarError: UndefVarError: `example_model` not defined

> **What if I Get An Error?**
>
> If `optimize!()` throws an error, that’s usually a sign that something
> is wrong with the formulation (for example, a variable might not be
> bounded or a constraint might not be specified correctly) or a typo in
> the model definition. Linear programs should be well behaved!

To find the values of variables after optimizing, use `value.()` (the
broadcasting ensures this will work for vector-valued variables as
well):

In [ ]:
value.(example_x)

In [ ]:
value.(example_z)

#### Problem 2.3 (1 point)

How would your profit change if you could buy 1,000 additional bf of
spruce? You can answer this by getting the shadow price of a particular
variable with:

In [ ]:
shadow_price(constraint1) # this is why we named the constraints when we defined them

#### Problem 2.4 (1 point)

Would you prefer to have 2,000 additional bf of spruce or 1,000
additional bf of fir?

In [83]:
#First I will define the wood model
extra_spruce_wood_model = Model(HiGHS.Optimizer)

#Next I will define the Spruce (S) and Fir (F) variables
T=1:3
@variable(extra_spruce_wood_model,S[t in T] >= 0)
@variable(extra_spruce_wood_model,F[t in T] >= 0)

#define the objective function
@objective(extra_spruce_wood_model, Max,400*((S[1]+F[1])/2000)
+520*((S[2]+F[2])/3000)
+700*((S[3]+F[3])/3500)
-sum(S)*0.12-sum(F)*0.08)

#define constraints
@constraint(extra_spruce_wood_model,Spruce_Resources,sum(S)<=(320000+2000))
    #added 2000 bf of spruce
@constraint(extra_spruce_wood_model,Fir_Resources,sum(F)<=720000)
@constraint(extra_spruce_wood_model,Grade1_ratio, F[1]==3*S[1])
@constraint(extra_spruce_wood_model,Grade2_ratio, F[2]==2*S[2])
@constraint(extra_spruce_wood_model,Grade3_ratio, F[3]==(4/3)*S[3])

optimize!(extra_spruce_wood_model)

return objective_value(extra_spruce_wood_model)


Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 3 cols, 6 nonzeros
2 rows, 3 cols, 6 nonzeros
Presolve : Reductions: rows 2(-3); columns 3(-3); elements 6(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.4666614604e-01 Ph1: 2(4.58333); Du: 3(0.446666) 0s
          2    -1.1216000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1216000000e+05
HiGHS run time      :          0.00


112160.00000000001

In [84]:
#First I will define the wood model
extra_fir_wood_model = Model(HiGHS.Optimizer)

#Next I will define the Spruce (S) and Fir (F) variables
T=1:3
@variable(extra_fir_wood_model,S[t in T] >= 0)
@variable(extra_fir_wood_model,F[t in T] >= 0)

#define the objective function
@objective(extra_fir_wood_model, Max,400*((S[1]+F[1])/2000)
+520*((S[2]+F[2])/3000)
+700*((S[3]+F[3])/3500)
-sum(S)*0.12-sum(F)*0.08)

#define constraints
@constraint(extra_fir_wood_model,Spruce_Resources,sum(S)<=320000)
@constraint(extra_fir_wood_model,Fir_Resources,sum(F)<=(720000+1000))
    #added 1000 bf of fir
@constraint(extra_fir_wood_model,Grade1_ratio, F[1]==3*S[1])
@constraint(extra_fir_wood_model,Grade2_ratio, F[2]==2*S[2])
@constraint(extra_fir_wood_model,Grade3_ratio, F[3]==(4/3)*S[3])

optimize!(extra_fir_wood_model)

return objective_value(extra_fir_wood_model)


Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 3 cols, 6 nonzeros
2 rows, 3 cols, 6 nonzeros
Presolve : Reductions: rows 2(-3); columns 3(-3); elements 6(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.4666614604e-01 Ph1: 2(4.58333); Du: 3(0.446666) 0s
          2    -1.1212000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1212000000e+05
HiGHS run time      :          0.00


112120.00000000001

I would prefer 2000 bf of extra Spruce as it optimizes with an objective value of $112,160 whereas an extra 1000 bf of extra fir results in $112,120 total revenue. The difference is an extra $40 if you choose the 2000 bf of spruce over 1000 bf of fir.

## References

Put any consulted sources here, including classmates you worked with/who
helped you.